## Building a Face Detection and Blurring Model

### Step-by-Step Guide

1. **Data Collection**
   - Use the [WIDER FACE dataset](http://shuoyang1213.me/WIDERFACE/).

2. **Model Architecture**
   - Design a Convolutional Neural Network (CNN) for face detection.

3. **Data Preprocessing**
   - Resize and normalize images.
   - Extract bounding boxes from annotations.

4. **Training**
   - Train the CNN model with the dataset.

5. **Face Detection and Blurring**
   - Detect and blur faces in new images using the trained model.

6. **Evaluation**
   - Evaluate the model using the validation set.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
def create_face_detection_model(input_shape=(128, 128, 3)):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(4))  # [x, y, width, height] for bounding box
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
model = create_face_detection_model()
model.summary()

In [ ]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET

In [ ]:
def load_data(data_dir, img_size=(128, 128)):
    images = []
    bboxes = []

    for subdir, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.jpg'):
                img_path = os.path.join(subdir, file)
                xml_path = os.path.join(subdir, file.replace('.jpg', '.xml'))
                
                image = cv2.imread(img_path)
                image = cv2.resize(image, img_size)
                image = image / 255.0
                
                tree = ET.parse(xml_path)
                root = tree.getroot()
                bbox = root.find('object').find('bndbox')
                
                xmin = int(bbox.find('xmin').text)
                ymin = int(bbox.find('ymin').text)
                xmax = int(bbox.find('xmax').text)
                ymax = int(bbox.find('ymax').text)
                
                width = xmax - xmin
                height = ymax - ymin
                
                # Normalize bounding box coordinates
                bbox = [xmin / img_size[0], ymin / img_size[1], width / img_size[0], height / img_size[1]]
                
                images.append(image)
                bboxes.append(bbox)
    
    return np.array(images), np.array(bboxes)

In [ ]:
X_train, Y_train = load_data('path_to_widerface_train')
X_val, Y_val = load_data('path_to_widerface_val')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = datagen.flow(X_train, Y_train, batch_size=32)
val_generator = datagen.flow(X_val, Y_val, batch_size=32)

In [ ]:
history = model.fit(train_generator, epochs=50, validation_data=val_generator)

In [ ]:
def blur_faces_in_image(model, image):
    input_image = cv2.resize(image, (128, 128))
    input_image = input_image / 255.0
    input_image = np.expand_dims(input_image, axis=0)
    
    bbox = model.predict(input_image)[0]
    x, y, w, h = bbox
    
    h, w, _ = image.shape
    x = int(x * w)
    y = int(y * h)
    width = int(w * w)
    height = int(h * h)
    
    face_region = image[y:y+height, x:x+width]
    face_region = cv2.GaussianBlur(face_region, (99, 99), 30)
    
    image[y:y+height, x:x+width] = face_region
    return image

In [ ]:
image = cv2.imread('add_your_path_here_uwu.jpg')

In [ ]:
blurred_image = blur_faces_in_image(model, image)

In [ ]:
cv2.imshow('Blurred Faces', blurred_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
val_loss = model.evaluate(val_generator)
print(f'Validation Loss: {val_loss}')

In [ ]:
model.save('face_detection_model.h5')
!mmtoir -f keras -w face_detection_model.h5 -o face_detection_model
!mmtocode -f caffe -n face_detection_model.pb -w face_detection_model.npy -o face_detection_model
!mmtomodel -f caffe -n face_detection_model.py -w face_detection_model.npy -o face_detection_model.caffemodel


Model is finished. Gets saved as face-detecting-model.caffemodel